In [18]:
import pandas as pd
import cobra
import driven
import numpy as np
import altair as alt
from driven.data_sets import ExpressionProfile
from driven import flux_analysis

ModuleNotFoundError: No module named 'driven.flux_analysis.gimme'

In [3]:
fname = '../data/nobori_2018/pnas.1800529115.sd02.xlsx'
conditions = ['MM','vivo','flg22','D36E','AvrRpt2','AvrRps4']
all_conditions_frames = []
condition_to_p = {}
# load the up and down sheet for each comparison, then merge
# them into a single dataframe and rename the columns to meet the
# conventions in the driven package.
for condition in conditions:
    down = pd.read_excel(fname,sheet_name=condition+"_down")
    down = down.rename({condition+'_down':'gene'},axis='columns')
    up = pd.read_excel(fname,sheet_name=condition+"_up")
    up = up.rename({condition+'_up':'gene'},axis='columns')
    both = pd.concat([down,up])
    fchange_col = both.columns
    expression_column = [c for c in both.columns if "fold change" in c][0]
    p_column = [c for c in both.columns if "q-value" in c][0]
    condition = expression_column.split(': ')[1]
    
    both = both.rename({p_column:\
        "p-value-"+condition,expression_column:condition},axis='columns')
    
    # split each gene name by '-' to get the KEGG gene ID only,
    # then reassign it to the index
    new_index = [s[0] for s in both['gene'].str.split('-')]
    both.index = new_index
    both = both.drop('gene',axis=1)
    all_conditions_frames.append(both)
    
    # save the exact condition name and the p value column associated with it
    condition_to_p[condition] = "p-value-"+condition
xomics_data = pd.concat(all_conditions_frames,axis=1)

# for p-value columns that are NaN after the merge, reassign as 1 to indicate
# not significant
p_cols = [c for c in xomics_data.columns if "p-value" in c]
xomics_data[p_cols] = xomics_data[p_cols].fillna(value=1.0)

# for expression, do the equivalent (set foldchange to 0)
diff_cols = [c for c in xomics_data.columns if "p-value" not in c]
xomics_data[diff_cols] = xomics_data[diff_cols].fillna(value=0.0)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31 FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



In [4]:
# Load the count data
fname = "../data/nobori_2018/GSE103441_count.txt"
count_frame = pd.read_csv(fname,sep='\t',index_col=0)
# remove sample names from the end of column names
# and merge samples group-wise into a mean
#'_'.join(count_frame.columns[0].split('_')[:-1])
groups = set(['_'.join(c.split('_')[:-1]) for c in count_frame.columns])
mean_count_frame = pd.DataFrame(index=count_frame.index)
for group in groups:
    samps = [col for col in count_frame.columns if group in col]
    mean = count_frame[samps].mean(axis=1)
    mean_count_frame[group] = mean
mean_count_frame

,Pto_pad4sid2,Pto_col_flg22,Pto_de,Pto-D36E_col,Pto_b2b3,Pto_pad4,Pto_col_mock,Pto_KB,Pto-AvrRpt2_KB,Pto-AvrRpt2_sid2pmr4,...,Pto_col,Pto_npr1,Pto-AvrRpt2_stp,Pto-D36E_KB,Pto-AvrRpt2_col,Pto-AvrRpt2_npr1,Pto-AvrRps4_col,Pto_MM_A80,Pto-AvrRpt2_pad4,Pto_stp
PSPTOA0002,0.00,0.0,0.00,0.000000,0.0,0.000,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0000,0.000000,0.0,0.00,0.000000,0.0
PSPTOA0004,0.00,0.0,0.00,0.000000,0.0,0.000,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0000,0.000000,0.0,0.00,0.000000,0.0
PSPTOA0005,0.00,0.0,0.00,0.000000,0.0,0.000,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0000,0.000000,0.0,0.00,0.000000,0.0
PSPTOA0007,0.00,0.0,0.00,0.000000,0.0,0.000,0.6,0.0,0.000000,0.0,...,0.103448,0.0,0.0,0.000000,0.0000,0.000000,0.0,0.00,0.000000,0.0
PSPTOA0008,0.00,0.0,0.00,0.000000,0.0,0.000,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0000,0.000000,0.0,0.00,0.000000,0.0
PSPTOA0009,0.00,0.0,0.00,0.000000,0.0,0.000,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0000,0.000000,0.0,0.00,0.000000,0.0
PSPTOA0010,0.00,0.0,0.00,0.000000,0.0,0.000,0.2,0.0,0.000000,0.0,...,0.034483,0.0,0.0,0.000000,0.0000,0.000000,0.0,0.00,0.000000,0.0
PSPTOA0011,0.00,0.0,0.00,0.000000,0.0,0.000,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0000,0.000000,0.0,0.00,0.000000,0.0
PSPTOA0012,0.00,0.0,0.00,0.000000,0.0,0.000,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0000,0.000000,0.0,0.00,0.000000,0.0
PSPTOA0013,0.00,0.0,0.00,0.000000,0.0,0.000,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0000,0.000000,0.0,0.00,0.000000,0.0


In [5]:
model = cobra.io.read_sbml_model("../results/reconstructions/pst7_complete_media.xml")
model.summary()

IN FLUXES            OUT FLUXES           OBJECTIVES
-------------------  -------------------  ------------
cpd00054_e    1e+03  cpd00011_e    1e+03  bio1  119
cpd00082_e    1e+03  cpd00035_e    1e+03
cpd00129_e  720      cpd00036_e    1e+03
cpd00106_e  698      cpd00067_e    1e+03
cpd10515_e  606      cpd00100_e    1e+03
cpd00130_e  457      cpd00477_e    1e+03
cpd00108_e  348      cpd10516_e  605
cpd00249_e  266      cpd00007_e  572
cpd00281_e  154      cpd00314_e  332
cpd00020_e   35.8    cpd00117_e  260
cpd00107_e   13.2    cpd00013_e   72
cpd00027_e    4.03   cpd00009_e   52.9
cpd00248_e    3.2    cpd00162_e    1.1
cpd00051_e    1.73
cpd00322_e    1.36
cpd00060_e    1.02
cpd00132_e    0.871
cpd00028_e    0.734
cpd00268_e    0.684
cpd00119_e    0.626
cpd00030_e    0.367
cpd00034_e    0.367
cpd00048_e    0.367
cpd00058_e    0.367
cpd00063_e    0.367
cpd00099_e    0.367
cpd00118_e    0.367
cpd00149_e    0.367
cpd00205_e    0.367
cpd00254_e    0.367
cpd00264_e    0.367
cpd01017_e    0

In [6]:
# extract the groups of interest from count frame
flg_v_mock_counts = mean_count_frame[['Pto_col_mock','Pto_col_flg22']]
flg_v_mm_counts = mean_count_frame[["Pto_col_flg22", "Pto_MM_A80"]]
# replace A in the gene names with _ as in the other
# spreadsheets provided by the author
#the following line is not necessary because PSPTO_0001 and PSPTOA0001 are different genes, changed to 
#replace "A" with "_A" like plasmid B
flg_v_mock_counts.index = flg_v_mock_counts.index.str.replace('A','_A')
flg_v_mm_counts.index = flg_v_mm_counts.index.str.replace('A', "_A")
# add p-values
flg_v_mock_p = xomics_data["p-value-flg22_Pto - Mock_Pto"]
flg_v_mock_counts['p-value'] = flg_v_mock_p
flg_v_mock_counts = flg_v_mock_counts.fillna(1)
#flg_v_mm_counts_p_val = xomics_data["p-value-"]

flg_v_mock_exp_prof = ExpressionProfile(identifiers=flg_v_mock_counts.index.tolist(),
                                        conditions=['Pto_col_mock','Pto_col_flg22'],
                                        expression=flg_v_mock_counts[['Pto_col_mock','Pto_col_flg22']].values,
                                        p_values=np.reshape(flg_v_mock_counts['p-value'].values,(-1, 1)))




/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [17]:
driven.flux_analysis.transcriptomics.gimme(model, expression_profile=flg_v_mock_exp_prof, cutoff=0.5)

NameError: name 'flux_analysis' is not defined

In [8]:
#threshold for the GIMME algorithm
#assign the variables to new ones
constrained_model, constrained_solution = driven.gimme(model,flg_v_mock_exp_prof,cutoff=5,fraction_of_optimum=0.05)
#open up all the exchange reactions, but gimme does not have OF of biomass,
#look into the gimme solution more, figure out how the driven model is modified and what is supposed to return
#gunna take some inkering, model that is further constrained than hte original by wither loss of reaction
#or changing the bounds

In [14]:
constrained_model.summary()

IN FLUXES            OUT FLUXES           OBJECTIVES
-------------------  -------------------  ------------
cpd00054_e    1e+03  cpd00011_e    1e+03  bio1  119
cpd00082_e    1e+03  cpd00035_e    1e+03
cpd00129_e  720      cpd00036_e    1e+03
cpd00106_e  698      cpd00067_e    1e+03
cpd10515_e  606      cpd00100_e    1e+03
cpd00130_e  457      cpd00477_e    1e+03
cpd00108_e  348      cpd10516_e  605
cpd00249_e  266      cpd00007_e  572
cpd00281_e  154      cpd00314_e  332
cpd00020_e   35.8    cpd00117_e  260
cpd00107_e   13.2    cpd00013_e   72
cpd00027_e    4.03   cpd00009_e   52.9
cpd00248_e    3.2    cpd00162_e    1.1
cpd00051_e    1.73
cpd00322_e    1.36
cpd00060_e    1.02
cpd00132_e    0.871
cpd00028_e    0.734
cpd00268_e    0.684
cpd00119_e    0.626
cpd00030_e    0.367
cpd00034_e    0.367
cpd00048_e    0.367
cpd00058_e    0.367
cpd00063_e    0.367
cpd00099_e    0.367
cpd00118_e    0.367
cpd00149_e    0.367
cpd00205_e    0.367
cpd00254_e    0.367
cpd00264_e    0.367
cpd01017_e    0

In [8]:
# try loading flg vs. mock expression first
flg_v_mock = xomics_data[["p-value-flg22_Pto - Mock_Pto","flg22_Pto - Mock_Pto"]]

In [9]:
flg_v_mock.loc[flg_v_mock['p-value-flg22_Pto - Mock_Pto'] < 0.01]

,p-value-flg22_Pto - Mock_Pto,flg22_Pto - Mock_Pto
PSPTO_0003,1.156187e-09,-1.057579
PSPTO_0005,1.863133e-05,-1.044168
PSPTO_0016,1.006172e-09,1.846849
PSPTO_0021,2.931064e-11,1.490298
PSPTO_0023,1.572790e-05,-1.374920
PSPTO_0025,9.026485e-07,1.700713
PSPTO_0043,2.475167e-14,1.639614
PSPTO_0044,8.589775e-07,-1.913224
PSPTO_0049,1.880575e-11,1.278185
PSPTO_0061,5.814111e-06,-1.159054


In [10]:
# How many differentially expressed genes are in the model, diff = significantly different genes
#big_diff = very significantly expressed genes
gene_ids = [gene.id for gene in model.genes]
diff = flg_v_mock.loc[flg_v_mock['p-value-flg22_Pto - Mock_Pto'] < 0.01]
diff_genes = diff.index
big_diff = diff.loc[abs(diff['flg22_Pto - Mock_Pto']) > 2]
big_diff_genes = big_diff.index
print("differentially expressed genes condition:",len(diff_genes))
print("Bigly-differentially expressed genes condition:",len(big_diff_genes))
print("differentially expressed genes in model:",len(set(gene_ids) & set(diff_genes)))
print("Bigly-differentially expressed genes in model:",len(set(gene_ids) & set(big_diff_genes)))

differentially expressed genes condition: 1292
Bigly-differentially expressed genes condition: 382
differentially expressed genes in model: 144
Bigly-differentially expressed genes in model: 41


In [11]:
#find the associated reactions to the differentially expressed genes in the diff genes lists
big_diff_in_model = set(gene_ids) & set(big_diff_genes)
associated_reactions = []
for gene in big_diff_in_model:
    for reaction in model.genes.get_by_id(gene).reactions:
        associated_reactions.append(reaction)

In [12]:
#generate the reactions associated as well as reaction ids
for reaction in associated_reactions:
    print(reaction.build_reaction_string())
    print (reaction.id)

cpd00002_c + cpd00010_c + cpd00024_c --> cpd00011_c + cpd00078_c
cor0003_c
cor00010_c + cpd00002_c + cpd00010_c --> cor00006_c
cor0007_c
cpd00003_c + cpd01795_c <=> cpd00004_c + cpd00067_c + cpd02441_c
rxn02842_c
cpd00001_c + cpd00344_c <=> cpd00013_c + cpd00770_c
rxn01640_c
cor00004_c + cpd00001_c --> cor00010_c + cpd11493_c
cor0006_c
cor00008_c + cpd00001_c --> cpd00067_c + cpd11406_c
cor0012_c
cpd00001_c + cpd00002_c + cpd00058_e --> cpd00008_c + cpd00009_c + cpd00058_c + cpd00067_c
rxn10481_c
cpd00001_c + cpd00002_c + cpd00058_c --> cpd00008_c + cpd00009_c + cpd00058_e + cpd00067_c
rxn05528_c
cpd00001_c + cpd00002_c + cpd04097_c --> cpd00008_c + cpd00009_c + cpd00067_c + cpd04097_e
rxn10344_c
cpd00001_c + cpd00002_c + cpd00531_c --> cpd00008_c + cpd00009_c + cpd00067_c + cpd00531_e
rxn10343_c
cpd00070_c + cpd00078_c + cpd11493_c --> cor00003_c
cor0004_c
cpd00035_c + cpd00287_c <=> cpd00020_c + cpd02096_c
rxn02925_c
cpd00035_c + cpd00191_c <=> cpd00020_c + cpd00085_c
rxn00656_c
cpd0

In [13]:
associated_reactions[0]

Reaction identifier,cor0003_c
Name,Coronafacic Acid Synthesis Reaction 1
Memory address,0x0125af9898
Stoichiometry,cpd00002_c + cpd00010_c + cpd00024_c --> cpd00011_c + cpd00078_c ATP + CoA + 2-Oxoglutarate --> CO2 + Succinyl-CoA
GPR,PSPTO_4680 or PSPTO_4685
Lower bound,0.0
Upper bound,1000.0


In [14]:
flg_v_mock.loc['PSPTO_4683']

p-value-flg22_Pto - Mock_Pto    1.826944e-28
flg22_Pto - Mock_Pto           -2.809649e+00
Name: PSPTO_4683, dtype: float64

In [15]:
#conditions = [key for key in condition_to_p.keys()]
#expression = xomics_data[]
flg_v_mock_exp_prof = ExpressionProfile(identifiers=flg_v_mock.index.tolist(),\
                                        conditions=['flg22_Pto - Mock_Pto'],\
                                        expression=np.matrix(flg_v_mock['flg22_Pto - Mock_Pto'].tolist()).transpose(),\
                                        p_values=np.matrix(flg_v_mock['p-value-flg22_Pto - Mock_Pto'].tolist()).transpose())

In [16]:
conditions = [key for key in condition_to_p.keys()]
xomics_profile = ExpressionProfile.from_data_frame(xomics_data[conditions])

In [20]:
print (flg_v_mock_exp_prof)

In [18]:
a = None
if not a:
    print("yep")

yep
